# **Tarefa 3 - Árvore de decisão**

####**Dicas:**

- Tutorial para iniciantes em Python: https://www.datacamp.com/cheat-sheet/getting-started-with-python-cheat-sheet

- Documentação do pandas: https://colab.research.google.com/drive/1a4sbKG7jOJGn4oeonQPA8XjJm7OYgcdX

- Documentação do scikit-learn: https://scikit-learn.org/stable/

- Documentação do matplotlib: https://matplotlib.org/stable/index.html

- Documentação do seaborn: https://seaborn.pydata.org/tutorial.html

---
## **Dataset:**

Nesse dataset os dados estão relacionados com campanhas de marketing direto (telefonemas) de uma instituição bancária portuguesa. O objetivo da classificação é prever se o cliente irá subscrever um depósito a prazo (**feature y**).


---


**Colunas:**

1 - **age**

2 - **education**

3 - **default**: tem crédito em atraso?

4 - **balance**: saldo médio anual, em euros

5 - **housing**: tem empréstimo para habitação?

6 - **loan**: tem empréstimo pessoal?

7 - **contact**: tipo de comunicação de contato

8 - **day**: último dia de contato do mês

9 - **month**: último mês de contato do ano

10 - **campaign**: número de contatos realizados durante esta campanha e para este cliente

11 - **previous**: número de contatos realizados antes desta campanha e para este cliente

12 - **poutcome**: resultado da campanha de marketing anterior

13 - **y** - o cliente subscreveu um depósito a prazo?

## **Importe os pacotes**

In [41]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split


## **Crie os datasets**
### **Crie um DataFrame a partir do arquivo de dados disponibilizados no EaD**

**Dicas:**

* Certifique-se que os arquivos a serem lidos estão carregados na sua sessão do Colab, ou em um drive previamente montado, e ajuste o comando para incluir todo o caminho até cada um dos arquivos a serem lidos.

In [42]:
df = pd.read_csv("bank.csv")
print(df.head())
print(df.education.value_counts())
print(df.contact.unique())
print(df.poutcome.value_counts())

df.describe()

   age  education default  balance housing loan   contact  day month  \
0   23  secondary      no      567     yes  yes  cellular   15   may   
1   35  secondary      no        0     yes   no  cellular   17   apr   
2   42  secondary      no      400     yes   no  cellular    3   feb   
3   35    primary      no      793     yes   no  cellular   17   apr   
4   45  secondary      no     1723      no   no  cellular    1   jun   

   campaign  previous poutcome    y  
0         1         0    other   no  
1         1         0    other   no  
2         5         6  failure   no  
3         1         0    other   no  
4         2         0    other  yes  
education
secondary    3967
tertiary     3055
primary      1012
unknown       356
Name: count, dtype: int64
['cellular' 'telephone']
poutcome
other      6181
failure    1166
success    1043
Name: count, dtype: int64


,age,balance,day,campaign,previous
count,8390.000000,8390.000000,8390.000000,8390.000000,8390.000000
mean,41.540167,1639.136949,15.557688,2.413468,1.050894
std,12.551213,3388.213746,8.389036,2.413275,2.510706
min,18.000000,-3058.000000,1.000000,1.000000,0.000000
25%,32.000000,139.000000,8.000000,1.000000,0.000000
50%,39.000000,592.000000,15.000000,2.000000,0.000000
75%,50.000000,1853.000000,22.000000,3.000000,1.000000
max,95.000000,81204.000000,31.000000,43.000000,58.000000


# **Pré-processamento**

Utilize o conhecimento adquirido nos exercícios anteriores para realizar o pré-processamento desta base de dados, incluindo normalização, tratamento de features categóricas e manipulação de valores nulos.

In [43]:
df[df['y'] == 'no']['education'].value_counts() # it's the same for both
df[df['education'] == 'unknown'] = 'secondary'

oe_education = OrdinalEncoder(categories = [['primary', 'secondary', 'tertiary']])
oe_month = OrdinalEncoder(categories = [[
    'january', 'february', 'march', 'april', 'may', 'june', 
    'july', 'august', 'september', 'october', 'november', 'december'
]])

df['education'] = oe_education.fit_transform(df['education'])
df['month'] = oe_month.fit_transform(df['month'])

le_default = LabelEncoder()
le_housing = LabelEncoder()
le_loan = LabelEncoder()
le_contact = LabelEncoder() # result of using OneHotEncoder would be the same, as there are only two possible values

df['default'] = le_default.fit_transform(df['default'])
df['housing'] = le_default.fit_transform(df['housing'])
df['loan'] = le_default.fit_transform(df['loan'])
df['contact'] = le_contact.fit_transform(df['contact'])

ohe = OneHotEncoder(sparse_output=False)




,age,education,default,balance,housing,loan,contact,day,month,campaign,previous,poutcome,y
0,23,secondary,no,567,yes,yes,cellular,15,may,1,0,other,no
1,35,secondary,no,0,yes,no,cellular,17,apr,1,0,other,no
2,42,secondary,no,400,yes,no,cellular,3,feb,5,6,failure,no
3,35,primary,no,793,yes,no,cellular,17,apr,1,0,other,no
4,45,secondary,no,1723,no,no,cellular,1,jun,2,0,other,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8385,60,tertiary,no,2222,yes,no,cellular,11,jul,2,0,other,yes
8386,39,secondary,no,150,no,no,cellular,3,jun,4,0,other,yes
8387,44,secondary,no,545,yes,no,cellular,6,apr,3,1,failure,no
8388,73,primary,no,1388,no,no,telephone,11,jan,2,1,success,yes


# **Criação de Conjuntos de Treinamento, Teste e Validação**

Divida os dados em conjuntos de treino, validação e teste. Para isso, selecione aleatoriamente 70% dos registros, sem reposição, para o conjunto de treinamento. Os 30% restantes deverão ser divididos igualmente entre os conjuntos de validação e teste.

Dicas:
- Fixe a semente de geração de dados aleatórios, utilize o comando `np.random.seed(escolha um número)`, antes de executar qualquer célula de comando que possa variar de valor resultante toda vez que for executada.

- Para fazer a divisão de treino, validação, teste use a função `train_test_split()`

In [4]:
np.random.seed(42)
X = df.drop('y', axis=1)
y = df['y']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5)

# **Treine e Teste o Modelo de Árvore de Decisão**

Treinar três versões diferentes do modelo de Árvore de Decisão utilizando valores variados para a sua profundidade.

**Dica:**

- Busque pela função `DecisionTreeClassifier(max_depth=?)` da biblioteca scikitlearn

# **Escolha e Avaliação do Modelo**

Neste momento, é importante avaliar cada um dos modelos gerados utilizando o **dataset de validação**. Apresente a matriz de confusão, bem como as métricas de acurácia, precisão, recall e F1-score para cada modelo. Após avaliá-los **escolha o melhor modelo**

**Dica:**

- Você pode usar a função `confusion_matrix(_, _)` da biblioteca `scikitlearn.metrics`
- Para calcular as métricas do modelo busque pelas funções `accuracy_score()`, `recall_score()`, `precision_score()`, `f1_score()`

# **Feature Importance**

Utilizando o melhor modelo gere o gráfico de importância das features para o modelo que obteve o melhor desempenho nos dados de validação. Em seguida, **remova as** **cinco** **features com menor importância e treine o modelo com o conjunto de treino novamente.**

**Dica**

- Utilize o método `.feature_importances_` da biblioteca `sklearn` para obter os valores de importância de cada feature.



# **Avaliação do Novo Modelo**

Com as cinco features removidas, avalie novamente o modelo de árvore de decisão escolhido, utilizando apenas o **conjunto de validação**.

# **Plotar a árvore do melhor modelo**

Gere a visualização da Árvore de Decisão do novo modelo

**Dica:**

- Use a função `plot_tree()` da biblioteca `sklearn`

- Para tornar a visualização mais clara e facilitar a interpretação dos resultados, ajuste o parâmetro max_depth em `plot_tree(max_depth=?)`, limitando a profundidade da árvore exibida e destacando os nós mais significativos.

# **Criação de Exemplos Fictícios**

Observe a estrutura da árvore de decisão treinada e crie dois exemplos fictícios, um para cada possível valor do target. Para cada exemplo, descreva detalhadamente o caminho que ele seguirá na árvore, ou seja, a sequência de nós que serão percorridos até chegar à folha correspondente.

# **Inferência e Verificação**

Realize uma inferência utilizando esses dois exemplos fictícios no modelo treinado. Verifique se os resultados obtidos na inferência correspondem aos valores do target que você imaginou ao criar os exemplos.

# **Escolha do melhor modelo**

Selecione o melhor modelo até agora com base no desempenho no conjunto de validação durante a etapa de avaliação e faça uma avaliação final do desempenho utilizando o **conjunto de** **teste**.

# **Mostre uma análise comparativa entre validação e teste para o modelo escolhido**

**Observação:** Se houver uma discrepância muito grande nos resultados, algo pode estar errado, verifique seu treinamento e avaliação do modelo.